En este notebook se hara el proceso de **PRIORIZAR** , se tomaran las PQR Desde diciembre del 2022 a  **2023 a Marzo(periodo de tres meses)**


# CONFIGURAR SPARK

In [1]:
import pandas as pd
import sys
sys.path.append('../librerias')


# LIBRERIAS





In [2]:
!python -m spacy download es_core_news_sm

!python -m spacy download es_core_news_lg

2023-07-07 14:44:42.721607: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-07 14:44:43.765467: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 49.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
2023-07-07 14:44:57.153155: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-07 14:44:58.323073: W tensorflow/compiler/tf2tensor

In [3]:
import pandas as pd

from google.colab import drive
import gc

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification




nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_sm', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing


#.master("local[2]")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [4]:
%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 33.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.0/425.0 kB 31.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycorrector: filename=pycorrector-0.5.0-py3-none-any.whl size=4480931 sha256=85cd8b4e7fa1b2b80b86c623a452f42dfd9b123990c1ef4347ca2e0cdcb30e48
  Stored in directory: /root/.cache/pip/wheels/03/cd/34/3c82eb07a21023abd3b8c334a96cb2b8f32cdd986d0f7d0bf4
  Created wheel for kenlm: filename=kenlm-0.1-cp310-cp310-linux_x86_64.whl size=3003892 sha256=0856279744d26a64f12262778edc1742388e688d6929659b97bb2fd9bfc39359
  Stored in directory: /root/.cache/pip/wheels/4e/3a/01/9105a071c30781823efbd96a58279c16f948a87cafb1144042
Successfully built pycorrector kenlm
     ━━━━━━━━━━━

# ETL

In [72]:

pandas_c_F= pd.read_csv('/content/PRIORIZAR/entranamiento_/binario_entrenamiento_cruda.csv',delimiter=';')

pandas_c_F= pandas_c_F.fillna(0.0)


pandas_c_F

,index,FECHA_CREACION,NUMERO_PQR,DESCRIPCION_PQR,clasificacion
0,0,22/02/23 9:19,MDM-PQR-37253010,ZULEIMA ALEZANDRA TORRES DIAZ 1031134671 CC ...,1.0
1,1,14/02/23 18:20,MDM-PQR-37091363,ZAIDA GUITIERREZ 52325972 CC 6013640557 3209...,0.0
2,2,3/12/22 8:51,MDM-PQR-35476275,YURY SUAREZ LLANES 51978002 CC 6012169386 3...,1.0
3,3,14/02/23 17:18,MDM-PQR-37089931,YULY MORENO 52381328 CC 6012529406 31678196...,0.0
4,4,14/02/23 21:44,MDM-PQR-37094750,YULY ANDREA GOMEZ ROMERO 1015467573 CC 314344...,0.0
...,...,...,...,...,...
15740,15740,21/02/23 17:11,MDM-PQR-37240623,LINEA NOMBRE JULY ROCIO RIANO RIANO CED...,1.0
15741,15741,10/01/23 13:01,MDM-PQR-36207108,NOMBRE: YENNY CEDULA: 51760212 LINE...,0.0
15742,15742,15/12/22 11:42,MDM-PQR-35702774,NOMBRE: EMILSEN CALDERON GARZON C...,1.0
15743,15743,25/02/23 15:29,MDM-PQR-37337458,"""Línea: Nombre de quien se comunica: ...",1.0


In [4]:
pandas_c_F.isnull().sum()

index              0
FECHA_CREACION     0
NUMERO_PQR         0
DESCRIPCION_PQR    0
clasificacion      0
dtype: int64

In [3]:
from clean_text import clean_text

entro2


In [4]:
#declare answers and questions
descripcion=pandas_c_F.DESCRIPCION_PQR	



# Cleaning the questions

clean_descripcion = []
for des in descripcion:
    clean_descripcion.append(clean_text(des))


#restar dos para optener la data de excel y poder comparar

clean_descripcion  [1002]

'usuairo queire valdidar  el saldo pendiente'

In [7]:
clean_descripcion  [15744]

''

In [8]:
from tqdm import tqdm
from spelling import correct_sentence

In [9]:
corrected_sentences = []
for text in tqdm(clean_descripcion, desc="Processing"):
    corrected_sentence = correct_sentence(text)
    corrected_sentences.append(corrected_sentence)

Processing: 100%|██████████| 15745/15745 [50:19<00:00,  5.21it/s]  


In [3]:
corrected_sentences1=corrected_sentences

In [5]:
corrected_sentences1  [933]

'ultima alexandra torres diaz cc c incremento en la factura'

In [ ]:
from lemastem import data_preprocessing


clean_descripcion=data_preprocessing(corrected_sentences1)


clean_descripcion[883]

In [13]:
clean_descripcion[4379]

['indicar', 'llego', 'factura', 'monto', 'pagar', 'normalmente']

In [14]:
import pickle



# Specify the file path where you want to save the list
file_path = "/content/PRIORIZAR/data/entrenamiento_binario.pkl"

# Open the file in binary write mode
with open(file_path, "wb") as file:
    # Use the pickle.dump() function to save the list to the file
    pickle.dump(clean_descripcion, file)

In [ ]:
import pickle

# Specify the file path from where you want to load the list
file_path = "/content/PRIORIZAR/data/entrenamiento_binario.pkl"

# Open the file in binary read mode
with open(file_path, "rb") as file:
    # Use the pickle.load() function to load the list from the file
    loaded_list = pickle.load(file)

print(loaded_list)

In [18]:

clean_descripcion=loaded_list

In [ ]:
len(clean_descripcion)

In [35]:
clean_descripcion[5668]

['saber',
 'factura',
 'brindar',
 'informar',
 'adjudicado',
 'descuento',
 'permanente',
 'bajo',
 'retirar',
 'incremento']

creacion tabla

In [73]:
import numpy as np
import pandas as pd
from unidecode import unidecode



merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion']).reset_index()

#############################################

pandas_c_F.reset_index(inplace=True)

#merged_df = pd.merge(merged_df, pandas_c_F, on='index', how='inner').loc[:, ['FECHA_CREACION','NUMERO_PQR','descripcion', 'clasificacion']]

merged_df = pd.concat([merged_df, pandas_c_F], axis=1)



shuffled_df = merged_df.sample(frac=1).reset_index(drop=True)




#shuffled_df.rename(columns={"FECHA_CREACION": "event_timestamp"}, inplace=True)

#string_values = ['abcde', 'fghij', 'klmno', 'pqrst', 'uvwxy', 'zabcd', 'efghi', 'jklmn', 'opqrs', 'tuvwx']



# Función para eliminar las tildes de un texto
def remove_accents(text):
    return unidecode(text)

# Aplicar la función a la columna 'Texto'
shuffled_df['descripcion'] = shuffled_df['descripcion'].apply(remove_accents)



# Randomly select string values to fill the DataFrame
#random_strings = np.random.choice(string_values, size=len(shuffled_df))

#shuffled_df["NUMERO_PQR"]=random_strings
shuffled_df['clasificacion'] = shuffled_df['clasificacion'].fillna(0.0)
shuffled_df['descripcion'] = shuffled_df['descripcion'].fillna("vacio")

shuffled_df

,index,descripcion,level_0,index,FECHA_CREACION,NUMERO_PQR,DESCRIPCION_PQR,clasificacion
0,6596,factura normalmente llegar mil pesos llego aho...,6596,6596,2/12/22 9:04,MDM-PQR-35454208,LINEA: SR-32435621 NOMBRE: JEREMI ALEXANDER M...,1.0
1,642,concepto facturar,642,642,22/02/23 11:41,MDM-PQR-37259220,WHATSAPP // JUAN SEBASTIAN CAMARGO TORRES // 1...,1.0
2,11000,gustar h pagar factura cada llego factura cost...,11000,11000,1/02/23 15:31,MDM-PQR-36766104,ESPERANZA GUZMAN CAMPOS 39523747 CC 60143659...,0.0
3,10147,beverly ranchero concepto facturar,10147,10147,3/01/23 7:14,MDM-PQR-36046354,JUVERLY LANCHEROS 1024499592 CC 6013101607 310...,1.0
4,5038,solicitar validacion concepto facturar,5038,5038,20/02/23 10:11,MDM-PQR-37197008,Nombre: JAIDER DAMIAN VILLALBA BONILLA Céd...,1.0
...,...,...,...,...,...,...,...,...
15740,15390,facturacion llego costo factura bajo desear sa...,15390,15390,18/02/23 9:25,MDM-PQR-37168548,LINEA NOMBRE ROSALBA FONSECA BARBOSA CEDULA...,1.0
15741,7158,revisar factura caer llamado,7158,7158,5/12/22 12:39,MDM-PQR-35510878,Linea: 6016314440 Nombre del cliente: Mauri...,0.0
15742,2221,gomezfacturacion,2221,2221,13/02/23 13:04,MDM-PQR-37054877,OLGA GOMEZ GOMEZ/facturacion,0.0
15743,15074,factura,15074,15074,27/01/23 8:24,MDM-PQR-36617424,NOMBRE: RICARDO JOSE ORTEGA VARGAS CEDULA: ...,0.0


In [76]:
shuffled_df.to_csv('/content/PRIORIZAR/entranamiento_/entrana_bina.csv', index=False)

In [70]:
shuffled_df = pd.read_csv('/content/PRIORIZAR/entranamiento_/entrana_bina.csv')

In [77]:
shuffled_df.isnull().sum()

index              0
descripcion        0
level_0            0
index              0
FECHA_CREACION     0
NUMERO_PQR         0
DESCRIPCION_PQR    0
clasificacion      0
dtype: int64

In [78]:

shuffled_df['clasificacion'].unique()

array([1., 0.])

In [79]:
from tfidf import TFIDF

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from joblib import dump



X = shuffled_df['descripcion']

y = shuffled_df['clasificacion']




# train test split (66% train - 33% test)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)




X_train,X_test = TFIDF(X_train,X_test)

pca = PCA()

pca.fit(X_train)

# Get the explained variance ratio for each component
explained_variance_ratio = pca.explained_variance_ratio_

# Calculate the cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# Find the number of components that explain a desired amount of variance
desired_variance = 0.9  # Set the desired variance threshold
n_components = np.argmax(cumulative_variance_ratio >= desired_variance) + 1

# Create a new instance of PCA with the optimal number of components
pca = PCA(n_components=n_components)


X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)



print("train with old features: ",np.array(X_train).shape)
print("train with new features:" ,np.array(X_train_new).shape)

print("test with old features: ",np.array(X_test).shape)
print("test with new features:" ,np.array(X_test_new).shape)



print('Training Data :', X_train.shape)

print('Testing Data : ', X_test.shape)


dump(value=pca, filename="/content/PRIORIZAR/modelo/pca_binario.joblib")

tf-idf with 3614 features
train with old features:  (12596, 3614)
train with new features: (12596, 804)
test with old features:  (3149, 3614)
test with new features: (3149, 804)
Training Data : (12596, 3614)
Testing Data :  (3149, 3614)


['/content/PRIORIZAR/modelo/pca_binario.joblib']

In [81]:
n_components

804

In [82]:
pip install optuna

In [83]:
import optuna
from optuna.samplers import TPESampler

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score


In [84]:
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        "verbosity": -1,
        "boosting_type": "gbdt",
        "n_estimators": trial.suggest_int("n_estimators", 80, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 5.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 5.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 30, 90),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        'learning_rate': trial.suggest_float("learning_rate", 0.04, 0.1),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }




    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train_new, y_train)
    preds = gbm.predict(X_test_new)
    accuracy = accuracy_score(y_test, preds)
    return accuracy




In [85]:
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightgbm", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)

[I 2023-07-07 15:39:28,112] A new study created in memory with name: lightgbm


[LightGBM] [Warning] bagging_fraction is set=0.4554031568612787, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4554031568612787
[LightGBM] [Warning] lambda_l2 is set=0.010007110473405195, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.010007110473405195
[LightGBM] [Warning] lambda_l1 is set=0.879294185249291, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.879294185249291
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.48805353449026784, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.48805353449026784


[I 2023-07-07 15:39:46,740] Trial 0 finished with value: 0.7939028262940616 and parameters: {'n_estimators': 172, 'lambda_l1': 0.879294185249291, 'lambda_l2': 0.010007110473405195, 'num_leaves': 48, 'feature_fraction': 0.48805353449026784, 'bagging_fraction': 0.4554031568612787, 'bagging_freq': 2, 'learning_rate': 0.060733643622582864, 'min_child_samples': 43}. Best is trial 0 with value: 0.7939028262940616.


[LightGBM] [Warning] bagging_fraction is set=0.4164325559187557, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4164325559187557
[LightGBM] [Warning] lambda_l2 is set=0.7069470359901728, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7069470359901728
[LightGBM] [Warning] lambda_l1 is set=0.1353297938831623, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1353297938831623
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9268704618345672, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9268704618345672


[I 2023-07-07 15:40:13,586] Trial 1 finished with value: 0.793585265163544 and parameters: {'n_estimators': 199, 'lambda_l1': 0.1353297938831623, 'lambda_l2': 0.7069470359901728, 'num_leaves': 42, 'feature_fraction': 0.9268704618345672, 'bagging_fraction': 0.4164325559187557, 'bagging_freq': 5, 'learning_rate': 0.06503828814202763, 'min_child_samples': 58}. Best is trial 0 with value: 0.7939028262940616.


[LightGBM] [Warning] bagging_fraction is set=0.8153935694015885, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8153935694015885
[LightGBM] [Warning] lambda_l2 is set=1.4493910392244416, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.4493910392244416
[LightGBM] [Warning] lambda_l1 is set=0.034250741406204634, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.034250741406204634
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.5880545068955457, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5880545068955457


In [ ]:
print('Best parameters:', study.best_params)

In [ ]:
mode = lgb.LGBMClassifier(**study.best_params)
mode.fit(X_train_new, y_train)

In [ ]:
y_pred = mode.predict(X_test_new)

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, classification_report

report = classification_report(y_test, y_pred)
print(report)

In [ ]:
from joblib import dump
dump(value=mode, filename="/content/PRIORIZAR/modelo/modelo_binario.joblib")

para data a predecir

modelo creacion